In [1]:
from bs4 import BeautifulSoup
import requests
import re
from re import search
import pandas as pd

# Cat Food URL

In [20]:
cat_food = []
for i in range(1,78):
    if i == 1:
        url = 'https://www.chewy.com/s?rh=c%3A325%2Cc%3A387'
    else:
        url = 'https://www.chewy.com/s?rh=c%3A325%2Cc%3A387&page='+str(i)
    session = requests.Session()
    response = session.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(response.text, "html5lib")
    page= soup.find('section', class_='results-products js-tracked-product-list')\
    .find_all('article',class_="product-holder js-tracked-product cw-card cw-card-hover")
    page_savings = soup.find('section', class_='results-products js-tracked-product-list')\
    .find_all('article',class_="product-holder js-tracked-product product-savings cw-card cw-card-hover")
    for link in page:
        cat_food.append(link)
    for link in page_savings:
        cat_food.append(link)
cat_food = [('https://www.chewy.com'+ link.find('a')['href'])for link in cat_food]

In [21]:
len(cat_food)

2743

# Dog Food URL

In [22]:
dog_food = []
for i in range(1,108):
    if i == 1:
        url = 'https://www.chewy.com/s?rh=c%3A288%2Cc%3A332'
    else:
        url = 'https://www.chewy.com/s?rh=c%3A288%2Cc%3A332&page='+str(i)
    session = requests.Session()
    response = session.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(response.text, "html5lib")
    page= soup.find('section', class_='results-products js-tracked-product-list')\
    .find_all('article',class_="product-holder js-tracked-product cw-card cw-card-hover")
    page_savings = soup.find('section', class_='results-products js-tracked-product-list')\
    .find_all('article',class_="product-holder js-tracked-product product-savings cw-card cw-card-hover")
    for link in page:
        dog_food.append(link)
    for link in page_savings:
        dog_food.append(link)
dog_food = [('https://www.chewy.com'+ link.find('a')['href'])for link in dog_food]

In [23]:
len(dog_food)

3852

# Cat Food DataFrame

In [1]:
def get_info(urls):

    df = pd.DataFrame(columns=['RecommendationScore', 'ReviewNum','ProductTitle', 'ItemNumber', 
                                      'Price','Weight_lbs','Brand','FoodForm','LifeStage','SpecialDiet',
                                     'Protein_min','Fat_min','Moisture_max','Fiber_max'])
    for url in cat_food:

        session = requests.Session()
        response = session.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        soup = BeautifulSoup(response.text, "html5lib")

        # number of reviews
        try:
            RevNum = soup.find('div', class_='ugc ugc-head').find('span', class_='hide-large').text
        except:
            RevNum = 0

        # recommendation score
        try:
            score = soup.find('span', class_='progress-radial__text--percent').text
            score = float(re.search('[\d\.]+',score).group(0))
        except:
            Score = 0

        # product title
        try:
            title = soup.find_all('title')[0].text
        except:
            title = None

        # list price
        try:
            price = soup.find(id = 'right-column').find(class_='product-pricing')\
            .find(class_='price').text.strip()
            price = float(re.search('[\d\.]+',price).group(0))
        except:
            price = None


        # item number
        try:
            ItemNum = soup.find('section',class_= 'descriptions__content cw-tabs__content--right')\
            .find('div',class_='value js-part-number').text.strip()
        except: None

        # life stage
        try:
            Lifestage = soup.find('section',class_= 'descriptions__content cw-tabs__content--right')\
            .find(text=re.compile('Lifestage')).findNext().text.strip()
        except:
            Lifestage = None

        # food form
        try:
            FoodForm = soup.find('section',class_= 'descriptions__content cw-tabs__content--right')\
            .find(text=re.compile('Food Form')).findNext().text.strip()
        except:
            FoodForm = None

        # special diet
        try:
            SpecialDiet = soup.find('section',class_= 'descriptions__content cw-tabs__content--right')\
            .find(text=re.compile('Special Diet')).findNext().text.strip()
        except:
            SpecialDiet = None

        # brand
        try:
            brand = soup.find('span',itemprop = 'brand').text
        except:
            brand = None

        # weight
        try:
            Weight = soup.find('section', class_= 'descriptions__content cw-tabs__content--right')\
            .find(text=re.compile('Weight'))
            weight_mess = Weight.findNext().text.strip()
            weight= re.sub('[^0-9\.(pounds)(ounces)]+',' ',weight_mess)
            if 'pounds' in weight:
                weight = float(re.search('[\d\.]+',weight).group(0))
            elif 'ounces' in weight:
                weight = float(re.search('[\d\.]+',weight).group(0))*0.0625
        except:
            weight = None

        # min protein
        try: 
            Nutrition = soup.find('article',id = 'Nutritional-Info')\
            .find('section', class_='cw-tabs__content--right').find_all('table')
            protein_average = 0
            protein_count = 0
            for i in Nutrition:
                protein = i.find(text=re.compile('Crude Protein')).next.text
                if 'max' not in protein:
                    protein = float(re.match('[\d\.]+',protein).group(0))
                    protein_average = protein_average + protein
                    protein_count += 1
            protein_average = protein_average/protein_count
        except:
            protein_average = None

        # min fat
        try:
            fat_average = 0
            fat_count = 0
            for i in Nutrition:
                fat = i.find(text=re.compile('Crude Fat')).next.text
                if 'max' not in fat:
                    fat = float(re.match('[\d\.]+',fat).group(0))
                    fat_average = fat_average + fat
                    fat_count += 1
            fat_average = fat_average/fat_count
        except:
            fat_average = None

        # max moisture
        try: 
            moisture_average = 0
            moisture_count = 0
            for i in Nutrition:
                moisture = i.find(text=re.compile('Moisture')).next.text
                if 'min' not in moisture:
                    moisture = float(re.match('[\d\.]+',moisture).group(0))
                    moisture_average = moisture_average + moisture
                    moisture_count += 1
            moisture_average = moisture_average/moisture_count
        except:
            moisture_average = None

        # max fiber
        try:
            fiber_average = 0
            fiber_count = 0
            for i in Nutrition:
                fiber = i.find(text=re.compile('Crude Fiber')).next.text
                if 'min' not in fiber:
                    fiber = float(re.match('[\d\.]+',fiber).group(0))
                    fiber_average =fiber_average + fiber
                    fiber_count += 1
            fiber_average = fiber_average/fiber_count
        except:
            fiber_average = None


        df = df.append({'RecommendationScore': score,
                                'ReviewNum': RevNum,
                                'ProductTitle': title,
                                'ItemNumber': ItemNum,
                                'Price': price,
                                'Weight_lbs': weight,
                                'Brand': brand,
                                'FoodForm': FoodForm,
                                'LifeStage': Lifestage,
                                'SpecialDiet': SpecialDiet,
                                'Protein_min': protein_average,
                                'Fat_min': fat_average,
                                'Moisture_max': moisture_average,
                                'Fiber_max': fiber_average},ignore_index=True)

In [ ]:
df_cat = get_info(cat_food)

# Dog Food DataFrame

In [ ]:
df_dog = get_info(dog_food)

# Save DataFrames

In [36]:
import pickle
with open('CatFoodList.pickle', 'wb') as to_write:
    pickle.dump(df_cat, to_write)

In [37]:
with open('DogFoodList.pickle', 'wb') as to_write:
    pickle.dump(df_dog, to_write)

In [40]:
df_dog.head(5)

,RecommendationScore,ReviewNum,ProductTitle,ItemNumber,Price,Weight_lbs,Brand,FoodForm,LifeStage,SpecialDiet,Protein_min,Fat_min,Moisture_max,Fiber_max
0,94.0,2676,American Journey Salmon & Sweet Potato Recipe ...,108423,42.99,24,American Journey,Dry Food,Adult,"Grain-Free, No Corn No Wheat No Soy, Gluten Fr...",32,14,10,5
1,95.0,295,Moist & Meaty Burger with Cheddar Cheese Flavo...,100187,15.99,13.5,Moist & Meaty,Dry Food,Adult,None,18,7,33,3
2,96.0,854,Blue Buffalo Life Protection Formula Small Bre...,46874,32.98,15,Blue Buffalo,Dry Food,Adult,No Corn No Wheat No Soy,26,15,10,4
3,100.0,19,American Journey Healthy Weight Salmon & Sweet...,183120,48.99,24,American Journey,Dry Food,Adult,"Grain-Free, No Corn No Wheat No Soy, Weight Co...",30,9,10,9
4,95.0,3310,Taste of the Wild High Prairie Grain-Free Dry ...,154551,48.99,None,Taste of the Wild,Dry Food,Adult,"Grain-Free, Gluten Free",32,18,10,4


In [41]:
df_cat.head(5)

,RecommendationScore,ReviewNum,ProductTitle,ItemNumber,Price,Weight_lbs,Brand,FoodForm,LifeStage,SpecialDiet,Protein_min,Fat_min,Moisture_max,Fiber_max
0,95.0,1244,Fancy Feast Gravy Lovers Poultry & Beef Feast ...,75978,16.79,5.53,Fancy Feast,Wet Food,Adult,None,9,2,82,1.5
1,96.0,826,Fancy Feast Grilled Seafood Feast Variety Pack...,76096,16.79,5.53,Fancy Feast,Wet Food,Adult,None,11,2,80,1.5
2,96.0,882,"Meow Mix Original Choice Dry Cat Food, 22-lb b...",99967,16.99,22,Meow Mix,Dry Food,Adult,None,31,11,12,4
3,95.0,873,Fancy Feast Poultry & Beef Classic Pate Variet...,76049,16.79,4.5,Fancy Feast,Wet Food,Adult,None,11,4.66667,78,1.5
4,96.0,973,"Friskies Shreds Variety Pack Canned Cat Food, ...",76368,18.50,12.93,Friskies,Wet Food,Adult,None,9,2.375,82,1
